<a href="https://colab.research.google.com/github/obara13/ml-test/blob/master/rnn_text_prediction_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNNを使ったテキスト生成

## 準備

In [0]:
import tensorflow as tf
import numpy as np
import os
import re
import time
import zipfile

青空文庫　源氏物語

In [0]:
file_dict = {
    'genzi01': 'https://www.aozora.gr.jp/cards/000052/files/5016_ruby_9746.zip',
    'genzi02': 'https://www.aozora.gr.jp/cards/000052/files/5017_ruby_9752.zip',
    'genzi03': 'https://www.aozora.gr.jp/cards/000052/files/5018_ruby_9754.zip',
    'genzi04': 'https://www.aozora.gr.jp/cards/000052/files/5019_ruby_9761.zip',
    'genzi05': 'https://www.aozora.gr.jp/cards/000052/files/5020_ruby_11253.zip',
    'genzi06': 'https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip',
    'genzi07': 'https://www.aozora.gr.jp/cards/000052/files/5022_ruby_10211.zip',
    'genzi08': 'https://www.aozora.gr.jp/cards/000052/files/5023_ruby_10174.zip',
    'genzi09': 'https://www.aozora.gr.jp/cards/000052/files/5024_ruby_11084.zip',
    'genzi10': 'https://www.aozora.gr.jp/cards/000052/files/5025_ruby_11635.zip',
}

data_dir = '/content/data/'
os.mkdir(data_dir)

for k, v in file_dict.items():
    tf.keras.utils.get_file(
        data_dir + k + '.zip',
        v,
    )
    with zipfile.ZipFile(data_dir + k + '.zip') as z:
        z.extractall(data_dir)


32768/29585 [=================================] - 0s 3us/step


In [0]:
file_list = [
    data_dir + '01kiritsubo.txt',
    data_dir + '02hahakigi.txt',
    data_dir + '03utsusemi.txt',
    data_dir + '04yugao.txt',
    data_dir + '05wakamurasaki.txt',
    data_dir + '06suetsumuhana.txt',
    data_dir + '07momijinoga.txt',
    data_dir + '08hanano_en.txt',
    data_dir + '09aoi.txt',
    data_dir + '10sakaki.txt',
]

本文以外を除去

In [0]:
text = ''
for f in file_list:
    t = open(f, 'rb').read().decode(encoding='shift-jis')
    t = re.sub(r'(.*-----)', '', t, flags=(re.DOTALL))
    t = re.sub(r'(底本：.*)', '', t, flags=(re.DOTALL))
    t = re.sub(r'(［＃.*字下げ］)', '', t)
    t = re.sub(r'(［＃.*］)', '', t)
    t = re.sub(r'《.*》', '', t)
    text += t

print(text)



紫のかがやく花と日の光思ひあはざる
ことわりもなし　　　　　　（晶子）

　どの天皇様の御代な家の娘たちにひけをとらせないよき保護者たりえた。それでも大官の後援者を持たぬ更衣は、何かの場合にいつも心細い思いをするようだった。
　前生した最初の女御であった。この女御がする批難と恨み言だけは無関心にしておいでになれなかった。この女御へ済まないという気も十分に持っておいでになった。帝の深い愛を信じながらも、悪く言う者と、何かの欠点を捜し出そうとする者ばかりの宮中に、病身な、そして無力な家を背景としている心細い更衣は、愛されれば愛されるほど苦しみがふえるふうであった。
　住んでいる御殿よりもまた深くなった。
　第二の皇子が三歳におなりになった時に袴着の生母になった更衣はこう呼ばれるのである――はちょっとした病気になって、実家へさがろうとしたが帝はお許しにならなかった。どこかからだが悪いということはこの人の常のことになっていたから、帝はそれほどお驚きにならずに、
「もうしばらく御所で養生をしてみてからにするがよい」
　と言っておいでになるうちにしだいに悪くなって、そうなってからほんの五、六日のうちに病は重体になった。母の未亡人は泣く泣くお暇を願って帰宅させることにした。こんな場合にはまたどんな呪詛いを及ぼしてはとの心づかいから、皇子だけを宮中にとどめて、目だたぬように御息所だけが退出するのであった。この上留めることは不可能であると帝は思召して、更衣が出かけて行くところを見送ることのできぬ御尊貴の御身の物足りなさを堪えがたく悲しんでおいでになった。
　はなやかな顔だちの美人が非常に痩を役人へお下しになったりあそばされても、また病室へお帰りになると今行くということをお許しにならない。
「死の旅にも同時に出るのがわれわれ二人であるとあなたも約束したのだから、私を置いて家へ行ってしまうことはできないはずだ」
　と、帝がお言いになると、そのお心持ちのよくわかる女も、非常に悲しそうにお顔を見て、


「限りとて別るる道の悲しきにいかまほしきは命なりけり


　死がそれほど私に迫って来ておりませんのでしたら」
　これだけのことを息も絶え絶えに言って、なお帝にお言いしたいことがありそうであるが、まったく気力はなくなってしまった。死ぬのであったらこのまま自分のそばで死なせたいと帝は思召も高

In [0]:
vocab = sorted(set(text))
print(len(text))
print(len(vocab))

121885
1510


リストを作成　文字→番号、番号→文字

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

print(char2idx)
print(idx2char)

{'\n': 0, '\r': 1, '―': 2, '\u3000': 3, '、': 4, '。': 5, '々': 6, '「': 7, '」': 8, '『': 9, '』': 10, 'あ': 11, 'い': 12, 'う': 13, 'え': 14, 'お': 15, 'か': 16, 'が': 17, 'き': 18, 'ぎ': 19, 'く': 20, 'ぐ': 21, 'け': 22, 'げ': 23, 'こ': 24, 'ご': 25, 'さ': 26, 'ざ': 27, 'し': 28, 'じ': 29, 'す': 30, 'ず': 31, 'せ': 32, 'ぜ': 33, 'そ': 34, 'ぞ': 35, 'た': 36, 'だ': 37, 'ち': 38, 'ぢ': 39, 'っ': 40, 'つ': 41, 'づ': 42, 'て': 43, 'で': 44, 'と': 45, 'ど': 46, 'な': 47, 'に': 48, 'ぬ': 49, 'ね': 50, 'の': 51, 'は': 52, 'ば': 53, 'ぱ': 54, 'ひ': 55, 'び': 56, 'ぴ': 57, 'ふ': 58, 'ぶ': 59, 'ぷ': 60, 'へ': 61, 'べ': 62, 'ほ': 63, 'ぼ': 64, 'ぽ': 65, 'ま': 66, 'み': 67, 'む': 68, 'め': 69, 'も': 70, 'ゃ': 71, 'や': 72, 'ゅ': 73, 'ゆ': 74, 'ょ': 75, 'よ': 76, 'ら': 77, 'り': 78, 'る': 79, 'れ': 80, 'ろ': 81, 'わ': 82, 'ゐ': 83, 'ゑ': 84, 'を': 85, 'ん': 86, '一': 87, '丁': 88, '七': 89, '万': 90, '丈': 91, '三': 92, '上': 93, '下': 94, '不': 95, '与': 96, '世': 97, '丞': 98, '両': 99, '並': 100, '中': 101, '主': 102, '久': 103, '乏': 104, '乗': 105, '九': 106, '乞': 107, '乱': 108, '乳': 109, '乾

本文の文字列を番号に変換

In [0]:
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int)
print(len(text_as_int))

[1 0 1 ... 0 1 0]
121885


In [0]:
print(text[:30])
print(text_as_int[:30])



紫のかがやく花と日の光思ひあはざる
ことわりもなし
[   1    0    1    0 1081   51   16   17   72   20 1163   45  712   51
  197  578   55   11   52   27   79    1    0   24   45   82   78   70
   47   28]


## 学習

In [0]:
seq_length = 64+1
examples_per_epoch = len(text) // seq_length

print(examples_per_epoch)

1875


In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length, drop_remainder=True)
for i in sequences.take(5):
    print(repr(''.join(idx2char[i.numpy()])))

'\r\n\r\n紫のかがやく花と日の光思ひあはざる\r\nことわりもなし\u3000\u3000\u3000\u3000\u3000\u3000（晶子）\r\n\r\n\u3000どの天皇様の御代な家の娘たちにひけをとら'
'せないよき保護者たりえた。それでも大官の後援者を持たぬ更衣は、何かの場合にいつも心細い思いをするようだった。\r\n\u3000前生した最初の女'
'御であった。この女御がする批難と恨み言だけは無関心にしておいでになれなかった。この女御へ済まないという気も十分に持っておいでになっ'
'た。帝の深い愛を信じながらも、悪く言う者と、何かの欠点を捜し出そうとする者ばかりの宮中に、病身な、そして無力な家を背景としている心'
'細い更衣は、愛されれば愛されるほど苦しみがふえるふうであった。\r\n\u3000住んでいる御殿よりもまた深くなった。\r\n\u3000第二の皇子が三歳にお'


input_text = 入力
target_text = 予測（入力を1文字後にずらす）

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in dataset.take(3):
    print(repr(''.join(idx2char[input_example.numpy()])))
    print(repr(''.join(idx2char[target_example.numpy()])))

'\r\n\r\n紫のかがやく花と日の光思ひあはざる\r\nことわりもなし\u3000\u3000\u3000\u3000\u3000\u3000（晶子）\r\n\r\n\u3000どの天皇様の御代な家の娘たちにひけをと'
'\n\r\n紫のかがやく花と日の光思ひあはざる\r\nことわりもなし\u3000\u3000\u3000\u3000\u3000\u3000（晶子）\r\n\r\n\u3000どの天皇様の御代な家の娘たちにひけをとら'
'せないよき保護者たりえた。それでも大官の後援者を持たぬ更衣は、何かの場合にいつも心細い思いをするようだった。\r\n\u3000前生した最初の'
'ないよき保護者たりえた。それでも大官の後援者を持たぬ更衣は、何かの場合にいつも心細い思いをするようだった。\r\n\u3000前生した最初の女'
'御であった。この女御がする批難と恨み言だけは無関心にしておいでになれなかった。この女御へ済まないという気も十分に持っておいでにな'
'であった。この女御がする批難と恨み言だけは無関心にしておいでになれなかった。この女御へ済まないという気も十分に持っておいでになっ'


データセットを作成

In [0]:
batch_size = 64
buffer_size = 10000

dataset = dataset.shuffle(buffer_size)
dataset = dataset.batch(batch_size, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 64), (64, 64)), types: (tf.int64, tf.int64)>

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

学習モデル
　embedding + rnn + 全結合層

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(
            vocab_size,
            embedding_dim,
            batch_input_shape = [batch_size, None]
        ),
        tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform',
        ),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [0]:
model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = batch_size
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           386560    
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 1510)          1547750   
Total params: 5,872,614
Trainable params: 5,872,614
Non-trainable params: 0
_________________________________________________________________


損失関数の定義

In [0]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
checkpoint_dir = '/content/ckpt/'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [0]:
epochs = 15
history = model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])

Epoch 1/15
29/29 [==============================] - 2s 65ms/step - loss: 5.3196
Epoch 2/15
29/29 [==============================] - 2s 63ms/step - loss: 4.4240
Epoch 3/15
29/29 [==============================] - 2s 63ms/step - loss: 3.8425
Epoch 4/15
29/29 [==============================] - 2s 64ms/step - loss: 3.5236
Epoch 5/15
29/29 [==============================] - 2s 64ms/step - loss: 3.3314
Epoch 6/15
29/29 [==============================] - 2s 64ms/step - loss: 3.1826
Epoch 7/15
29/29 [==============================] - 2s 64ms/step - loss: 3.0559
Epoch 8/15
29/29 [==============================] - 2s 64ms/step - loss: 2.9457
Epoch 9/15
29/29 [==============================] - 2s 68ms/step - loss: 2.8433
Epoch 10/15
29/29 [==============================] - 2s 64ms/step - loss: 2.7429
Epoch 11/15
29/29 [==============================] - 2s 65ms/step - loss: 2.6506
Epoch 12/15
29/29 [==============================] - 2s 64ms/step - loss: 2.5553
Epoch 13/15
29/29 [==================

## 予測

予測時の入力は batch_size = 1 となるので、モデルを再作成して学習データを読み込む

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            386560    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 1510)           1547750   
Total params: 5,872,614
Trainable params: 5,872,614
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
    num_generate = 64
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


In [61]:
print(generate_text(model, start_string='源氏は'))

源氏は泣告えた。優しい源氏の宰通の女に加わったこんな女のほさらを覚えた。


こちつや見せな態度は涙になっていなって、何かも明け
